# Depth Image Transformations

In [1]:
import numpy as np
import math
import cv2
import os, os.path
import sys
import time
import pandas as pd

path = 'C:/Users/jumpr_000/Desktop/ransac_obstacle_detection/saved' #set to your desired path to save images in
frames_dir = "kinect_data/" #the directory holding the saved depth images in .npy format

#camera information based on the Kinect v2 hardware
CameraParams = {
  "cx":254.878,
  "cy":205.395,
  "fx":365.456,
  "fy":365.456,
  "k1":0.0905474,
  "k2":-0.26819,
  "k3":0.0950862,
  "p1":0.0,
  "p2":0.0,
}

# Kinect's physical orientation in the real world.
CameraPosition = {
    "x": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "y": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "z": 0, # height in meters of actual kinect sensor from the floor.
    "roll": 0, # angle in degrees of sensor's roll (used for INU input - trig function for this is commented out by default).
    "azimuth": 0, # sensor's yaw angle in degrees.
    "elevation": -30, # sensor's pitch angle in degrees.
}

def depthMatrixToPointCloudPos(z, scale=1000):
    """
    xyz_arr = depthMatrixToPointCloudPos(depth_frame)

    Given a depth image, converts each
    point to an XYZ coordinate and returns
    an array of these points.
    The scale is a conversion factor.
    Default converts from milimeter
    depth data to meters.
    """
    C, R = np.indices(z.shape)

    R = np.subtract(R, CameraParams['cx'])
    R = np.multiply(R, z)
    R = np.divide(R, CameraParams['fx'] * scale)

    C = np.subtract(C, CameraParams['cy'])
    C = np.multiply(C, z)
    C = np.divide(C, CameraParams['fy'] * scale)

    return np.column_stack((z.ravel() / scale, R.ravel(), -C.ravel()))

def depthToPointCloudPos(x_d, y_d, z, scale=1000):
    """
    x, y, z = (row, col, depth)
    
    Given a point from a depth image
    and its position in the image,
    returns the x, y, and z coordinates
    relative to the camera location.
    The scale is a conversion factor.
    Default converts from milimeter
    depth data to meters.
    """
    x = (x_d - CameraParams['cx']) * z / CameraParams['fx']
    y = (y_d - CameraParams['cy']) * z / CameraParams['fy']

    return x / scale, y / scale, z / scale

def applyCameraOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # This runs slowly in Python as it is required to be called within a loop, but it is a more intuitive example than it's vertorized alternative (Purly for example)
    # use trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[ax1] ** 2 + pt[ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[ax2], pt[ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(0, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(1, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y plane

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

def applyCameraMatrixOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # bacically this is a vectorized version of applyCameraOrientation()
    # uses same trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[:, ax1] ** 2 + pt[:, ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[:, ax2], pt[:, ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[:, ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[:, ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(1, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(0, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

# Plane Fitting

In [2]:
def plane_fit(points):
    """
    p, n = plane_fit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    from numpy.linalg import svd
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

def get_orientation(xyz_arr, n_iter):
    """
    center, plane, theta = get_orientation(xyz_arr, num_points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fits a plane to the array n_iter times and
    returns the average of the planes centers,
    norms, and pitch degrees.
    """
    pitch = []
    planes = []
    centers = []
    for _ in range(0, n_iter):
        rand_points = []
        while len(rand_points) < 10:
            index = random.randrange(0,len(xyz_arr))
            if not xyz_arr[index].all() == np.zeros(3).all():
                rand_points.append(xyz_arr[index])
        rand_points = np.array(rand_points).T
        ctr, P = plane_fit(rand_points)
        r = math.sqrt(P[0]**2 + P[1]**2 + P[2]**2)
        theta = math.acos(P[2]/r) * 180 / math.pi
        phi = math.atan(P[1]/P[0]) * 180 / math.pi
        pitch.append(theta)
        planes.append(P)
        centers.append(ctr)
    return np.mean(centers, axis = 0), np.mean(planes, axis = 0), np.mean(pitch)
    

# Obstacle Detection

In [3]:
import numpy as np
import math
import random
import cv2
import os, os.path
import sys
import time
import pandas as pd
from pyntcloud import PyntCloud
from IPython.display import display, HTML

def get_obstacles_with_plane(depth_frame, num_planes, num_points, dist_thresh, visualize):
    obstacles = np.zeros(depth_frame.shape) #empty image that will store the locations of detected obstacles
    img = np.uint8(depth_frame) #some opencv functions require a byte image

    xyz_arr = depthMatrixToPointCloudPos(depth_frame) #convert depth data to XYZ coordinates
    start_time = time.time()
    center, plane, theta = get_orientation(xyz_arr, num_points)
    CameraPosition['elevation'] = -theta
    center = applyCameraOrientation(center)
    plane = applyCameraOrientation(plane)
    xyz_arr = applyCameraMatrixOrientation(xyz_arr)
    plane_img = np.zeros(len(xyz_arr))
    plane_img[xyz_arr[:,2] > dist_thresh - center[2]] = 1
    if visualize:
        xyz_arr = xyz_arr[xyz_arr[:,2] > dist_thresh - center[2]]
        points = pd.DataFrame(xyz_arr, columns=['x', 'y', 'z']) #convert XYZ coordinates to a DataFrame for pyntcloud
        cloud = PyntCloud(points)

        lines = [
            {
                # X axis
                "color": "red", 
                "vertices": [[0, 0, 0], [10, 0, 0]]
            },
            {
                # Y axis
                "color": "green",
                "vertices": [[0, 0, 0], [0, 10, 0]]
            },
            {
                # Z axis
                "color": "blue",
                "vertices": [[0, 0, 0], [0, 0, 10]]
            },
            {
                #Original norm of the plane
                "color": "pink",
                "vertices": [center.tolist(), plane.tolist()]
            }
        ]
        cloud.plot(cmap="cool", polylines=lines)

    plane_img = np.uint8(np.reshape(plane_img,(424,512)) * 255) #reshape to match depth data and convert to uint8
    plane_img = np.uint8((np.ones((424,512)) * 255) - plane_img) #invert img so pixel value corresponds to NOT ground plane
    ret, plane_img = cv2.threshold(plane_img,0,255,cv2.THRESH_BINARY) #filter points that are probaly not ground plane
    plane_img = cv2.subtract(img, plane_img)
    
    #noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(plane_img ,cv2.MORPH_OPEN, kernel, iterations = 3) #erosion followed by dilation

    color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) #BGR image to draw labels on

    #begin contour detection
    image, contours, hierarchy = cv2.findContours(opening,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    color = cv2.drawContours(color, contours, -1, (0,255,0), 1)
    for cntr in contours:
        try:
            #calculate diamter of equivalent circle
            #this measurement is only used for checking if countours fit our bounds
            area = cv2.contourArea(cntr)
            equi_diameter = np.sqrt(4*area/np.pi)

            #Hardcoded Diameter Range in pixels
            LOW_DIAMETER_BOUND = 20
            HIGH_DIAMETER_BOUND = 150

            HIGH_DISTANCE_BOUND = 4500
            #Original tolerances were 20 and 150

            if(equi_diameter>LOW_DIAMETER_BOUND and equi_diameter<HIGH_DIAMETER_BOUND):
                mask = np.zeros_like(img) #mask will contain the fitted and adjusted ellipse of a single obstacle
                ellipse = cv2.fitEllipse(cntr)
                x,y,obj_length,obj_height = cv2.boundingRect(cntr)
                rect = cv2.minAreaRect(cntr)

                equi_diameter = obj_length #bounding rectangle gives a better approximation of diameter

                box = cv2.boxPoints(rect)
                box = np.int0(box)
                mask = cv2.ellipse(mask,ellipse,(255,255,255),-1) #draw the fitted ellipse
                rows = mask.shape[0]
                cols = mask.shape[1]
                M = np.float32([[1,0,0],[0,1,equi_diameter/4]]) #shift mask down to match obstacle, not edge
                mask = cv2.warpAffine(mask,M,(cols,rows))
                mask = cv2.erode(mask, kernel, iterations=3) #erode the mask to remove background points
                img_fg = cv2.bitwise_and(depth_frame,depth_frame,mask = mask) #use the mask to isolate original depth values
                img_fg = cv2.medianBlur(img_fg,5) #median blur to further remove noise
                obstacles = cv2.add(np.float32(img_fg), np.float32(obstacles)) 
                
                mean_val = np.median(img_fg[img_fg.nonzero()]) #compute the non-zero average of obstacle depth values
                min_val, distance_to_object, min_loc, max_loc = cv2.minMaxLoc(img_fg)

                moment = cv2.moments(cntr) #get the centroid of the obstacle using its moment
                cx = int(moment['m10']/moment['m00'])
                cy = int(moment['m01']/moment['m00'])

                if mean_val < HIGH_DISTANCE_BOUND: #kinect loses accuracy beyond 4.5m
                    coords = depthToPointCloudPos(cx, cy, mean_val) #convert obstacle depth to XYZ coordinate

                    mm_diameter = (equi_diameter) * (1.0 / CameraParams['fx']) * mean_val #convert pixel diameter to mm

                    #begin visualization
                    cv2.drawContours(color,[box],0,(0,0,255),1)
                    cv2.rectangle(color,(x,y),(x+obj_length,y+obj_height),(0,255,0),2)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(color, "x" + str(coords[0]), (cx,cy+30), font, 0.4, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.putText(color, "y" + str(coords[1]), (cx,cy+45), font, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.putText(color, "z" + str(mean_val), (cx,cy+60), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(color,"diameter = " + str(mm_diameter), (cx,cy + 15), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

        except:
            print ("Failed to process image")
            print (sys.exc_info()[0])
            
    elapsed_time = time.time() - start_time
    print("Frame took " + str(elapsed_time) + " seconds to process")

    cv2.imshow("plane",plane_img)
    cv2.imshow("img",depth_frame)
    cv2.imshow("final",color)
    cv2.imshow("sure_bg",opening)
    cv2.imshow("obstacles",obstacles)
    key = cv2.waitKey(delay=1)
    return color



# Single Image Testing

In [4]:
img = np.load('kinect_data/116.npy')
output = get_obstacles_with_plane(img, num_planes = 46, 
                                  num_points = 45, 
                                  dist_thresh = 0.1, 
                                  visualize = True)

C:\Users\jumpr\Anaconda2\envs\py36\lib\site-packages\pythreejs\traits.py:175: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(0.0054974926876575295, 0.48841281082603055, …

Frame took 0.6858265399932861 seconds to process


# Multiple Frame Testing

In [6]:
frames_dir = 'kinect_data/'
directory = os.fsencode(frames_dir)
frame_i = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".npy"):
        file_path = frames_dir + filename
        print(file_path)
        img = np.load(file_path)
        try:
            output = get_obstacles_with_plane(img, num_planes = 46, 
                                  num_points = 45, 
                                  dist_thresh = 0.1, 
                                  visualize = False)
            cv2.imwrite(os.path.join(path ,filename[:-4] + '.png'), output)
            frame_i += 1
        except:
            print("Could not find obstacles")
cv2.destroyAllWindows()

C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead
  after removing the cwd from sys.path.


kinect_data/0.npy
Frame took 0.06995749473571777 seconds to process
kinect_data/1.npy
Frame took 0.07194018363952637 seconds to process
kinect_data/10.npy
Frame took 0.06596136093139648 seconds to process
kinect_data/100.npy
Frame took 0.0859670639038086 seconds to process
kinect_data/101.npy
Frame took 0.08493208885192871 seconds to process
kinect_data/102.npy
Frame took 0.07595276832580566 seconds to process
kinect_data/103.npy
Frame took 0.08395123481750488 seconds to process
kinect_data/104.npy
Frame took 0.08195304870605469 seconds to process
kinect_data/105.npy
Frame took 0.08296537399291992 seconds to process
kinect_data/106.npy
Frame took 0.08494925498962402 seconds to process
kinect_data/107.npy
Frame took 0.08394265174865723 seconds to process
kinect_data/108.npy
Frame took 0.08393621444702148 seconds to process
kinect_data/109.npy
Frame took 0.08494710922241211 seconds to process
kinect_data/11.npy
Frame took 0.07192826271057129 seconds to process
kinect_data/110.npy
Frame t

Frame took 0.07595562934875488 seconds to process
kinect_data/205.npy
Frame took 0.06496214866638184 seconds to process
kinect_data/206.npy
Frame took 0.06997156143188477 seconds to process
kinect_data/207.npy
Frame took 0.06894111633300781 seconds to process
kinect_data/208.npy
Frame took 0.06894373893737793 seconds to process
kinect_data/209.npy
Frame took 0.07294201850891113 seconds to process
kinect_data/21.npy
Frame took 0.07193922996520996 seconds to process
kinect_data/210.npy
Frame took 0.06895852088928223 seconds to process
kinect_data/211.npy
Frame took 0.06894063949584961 seconds to process
kinect_data/212.npy
Frame took 0.07096362113952637 seconds to process
kinect_data/213.npy
Frame took 0.06894183158874512 seconds to process
kinect_data/214.npy
Frame took 0.06897830963134766 seconds to process
kinect_data/215.npy
Frame took 0.06796073913574219 seconds to process
kinect_data/216.npy
Frame took 0.07295513153076172 seconds to process
kinect_data/217.npy
Frame took 0.06994128